셀리니움은 브라우져를 자동화하는 툴로 테스팅을 목적으로 웹 어플리케이션을 자동화하는 목적으로 만들어졌지만 테스팅에 국한되서 사용되지는 않는다. 웹 기반의 작업들을 자동화하여 업무 효율성을 높일 수 있으며 다양한 언어와 다양한 브라우져를 지원한다.

In [55]:
from selenium import webdriver
import time
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
driver = webdriver.Chrome()
# make browser

###### move to naver website
driver.get('http://naver.com')

In [4]:
driver.set_window_size(800,600)

In [5]:
# 브라우저의 스크룰 위치 이동은 selenium webdriver 안에 메소드가 따로 없기 때문에 js 코드를 실행하여 
# 실행할 수 있다.-- execute_script method
script = 'window.scrollTo(200,300)'
driver.execute_script(script)

In [6]:
# driver 의 현재 윈도우 객체를 저장한다. 이와 같이 윈도우의 객체를 저장하면 브라우져에
# 여러 개의 탭이 있을 때 다른 탭에서 저장되어 있는 윈도우 객체의 탭으로 돌아올 수 있다.
main_window = driver.current_window_handle
main_window

'CDwindow-BC060A23508DB00B8DA53628D2ED7435'

In [7]:
# new tap opens
script = 'window.open("https://google.com")'
driver.execute_script(script)

In [15]:
# alert 다루기
# 아래와 같이 alert가 나오는 js code를 실행시키면 크롬 브라우져의 웹 페이지에서 alert메시지가 나옵니다. 
script = 'alert("selenium test");'
driver.execute_script(script)

In [24]:
driver.execute_script("alert('selenium test');")
alert = driver.switch_to_alert()
alert.text
time.sleep(2)
alert.accept()

confirm은 alert와 비슷하지만 취소버튼도 있다. selenium 에서는 confirm의 확인,취소버튼을 누를 수 있는 기능을 제공한다. 

In [27]:
script = 'confirm("confirm?");'
driver.execute_script(script)

In [28]:
alert = driver.switch_to_alert()
alert.text

'confirm?'

In [30]:
alert.accept() #ok button
alert.dismiss()# cancel button
driver.close() #close the current window browser
driver.quit()# quit the browser

google vision api file uploading practice

In [31]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get('https://cloud.google.com/vision/')

google vision api에서 파일을 업로드하는 영역은 iframe으로 되어 있다. 웹페이지 안에 다른 url을 가지는 웹페이지를 보여주는 기능이 있는 태그를 iframe이라고한다. 우선 iframe으로 포커싱을 해주어야 한다.

In [33]:
iframe = driver.find_element_by_css_selector('#vision_demo_section > iframe')
driver.switch_to_frame(iframe)

In [34]:
# driver.switch_to_default_content
# 위의 함수를 쓰면 포커싱이 iframe에서 나와 사이트를 다시 가리키게 됩니다.

In [35]:
# file upload
# you should write the path of the file
file_path = '/home/hskimim/Downloads/dog.png'
driver.find_element_by_css_selector('#input').send_keys(file_path)

In [36]:
driver.find_element_by_css_selector('#webDetection').click()

In [41]:
web_results = driver.find_elements_by_css_selector('#card > .container  > .row')[:15]
for web_result in web_results:
    print(
    web_result.find_element_by_css_selector('.name').text,
    web_result.find_element_by_css_selector('.score').text
    )

French Bulldog 1.29047
Boston Terrier 1.28066
Bulldog 1.27885
Bull Terrier 1.07139
Soft-coated Wheaten Terrier 0.98682
Puppy 0.8898
Border Collie 0.78608
Terrier 0.6546
Pet 0.6301
Brindle 0.5639
Dog breed 0.4977
Breed 0.4297
Veterinarian 0.4175
Black Brindle 0.4112
Dog 0


In [44]:
driver.find_element_by_css_selector('#fullTextAnnotation').click()

In [45]:
doc_results = driver.find_elements_by_css_selector('.page > .container.style-scope.vs-document > fieldset')
len(doc_results)

2

In [46]:
documents = []
for doc_result in doc_results:
    words = doc_result.find_elements_by_css_selector('.container .word')
    sentence_list = []
    for word in words:
        word = word.text.replace(' ','')
        sentence_list.append(word)
    documents.append(' '.join(sentence_list))
documents

['In the dog house', "' 인기를 잃다 , 면목을 잃다"]

In [47]:
driver.quit()

### 스크래핑
- gmarket 베스트 셀러 데이터를 수집

In [48]:
url = 'http://corners.gmarket.co.kr/Bestsellers'
driver = webdriver.Chrome()
driver.get(url)

In [60]:
best_commodities = driver.find_elements_by_css_selector('.best-list:nth-child(3)>ul >li ')
len(best_commodities)

200

In [72]:
dict_list=[]
for commodity in best_commodities[:100]:
    dict_list.append({
        'title':commodity.find_element_by_css_selector('.itemname').text,
        'o-price':commodity.find_element_by_css_selector('.o-price')\
        .text.split(" ")[0].replace(',',"")[:-1]
        's-price':commodity.find_element_by_css_selector('.s-price').split(" ")[0].replace(',',"")[:-1],
        'link':commodity.find_element_by_css_selector('.itemname').get_attribute('href')
    })
df = pd.DataFrame(dict_list)
df

SyntaxError: invalid syntax (<ipython-input-72-6648ff33cec7>, line 6)

In [68]:
is_nodata=df['o-price']==''
df['o-price'][is_nodata]

25    
44    
89    
Name: o-price, dtype: object

In [69]:
df['o-price'][is_nodata] = df['s-price'][is_nodata]
df

,link,o-price,s-price,title
0,http://item.gmarket.co.kr/Item?goodscode=62724...,"19,500원","5,900원 69%",더다다/원피스형 롱티/신상/루즈/박시/빅사이즈
1,http://item.gmarket.co.kr/Item?goodscode=14301...,"39,000원","28,470원 27%",[아디다스]니짜 CQ2332외 4종 택1 G/ 아디다스
2,http://item.gmarket.co.kr/Item?goodscode=63617...,"30,000원","12,900원 57%",[동원샘물미네마인]동원 샘물 2Lx9병x3팩(총 27병) /생수
3,http://item.gmarket.co.kr/Item?goodscode=41037...,"24,000원","4,900원 79%",[아망떼]1+1여름베개커버/인견/순면/30x50/40x60/50x70
4,http://item.gmarket.co.kr/Item?goodscode=14365...,"39,900원","32,900원 17%",[하기스]2018년형 에어솔솔 썸머 팬티 / 밴드 기저귀 2팩
5,http://item.gmarket.co.kr/Item?goodscode=13005...,"20,000원","12,900원 35%",[하림]하림 프로라인 치킨너겟 1kg 2봉+치킨너겟 100g(증정)
6,http://item.gmarket.co.kr/Item?goodscode=87359...,"30,000원","13,900원 53%",[동원덴마크우유]동원 인포켓치즈 20g x20개+(저키x8개 or 치즈디퍼즈)
7,http://item.gmarket.co.kr/Item?goodscode=86949...,"29,600원","8,900원 69%",유스타일팜 여름신상/빅사이즈/밴딩/데님/스커트
8,http://item.gmarket.co.kr/Item?goodscode=91202...,"27,100원","16,900원 37%",CJ 햇반210gx24개/오뚜기밥24개/컵밥10개/컵반
9,http://item.gmarket.co.kr/Item?goodscode=14298...,"29,000원","14,900원 48%",나이키 스우시 라운드 반팔 티셔츠


In [ ]:
ratio = (df['o-price'].astype('int')-df['s-price'].astype('int'))/df['o-price'].astype('int')
ratio = (ratio*100).astype('int').astype('str')+'%'
df['ratio']=ratio
df

In [ ]:
driver.quit()